In [16]:
import json
import urllib
import httplib2 as http 

In [26]:
link = "http://datamall2.mytransport.sg/ltaodataservice/BusArrivalv2?BusStopCode=83139"

headers = {"AccountKey": "",                # LTA Datamall API Key
           "accept": "application/json"}

target = urllib.parse.urlparse(link)
method = "GET"
body = ""

h = http.Http()

response, content = h.request(
    target.geturl(),
    method,
    body,
    headers
)

#jsonObj = json.loads(content)
#print(json.dumps(jsonObj, sort_keys=True, indent=3))
